In [405]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from splinter import Browser
from io import StringIO
import time
import requests
import datetime as dt
import openpyxl
from openpyxl import workbook
from openpyxl import load_workbook

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common import keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import warnings
warnings.filterwarnings("ignore")

In [406]:
#install new Chrome webdriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Will_Boisseau\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [411]:
# Scrape Morningstar

# Create webdriver
driver=webdriver.Chrome()
driver.get("https://www.morningstar.com/etfs/xnas/acwi/portfolio")
#print(value)

# Close the browser after scraping
#driver.quit()


In [408]:
time.sleep(2)

In [414]:
#full xpath: /html/body/div[2]/div/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/div/sal-components-funds-portfolio/div/div[1]/div[2]/div/div[6]/div[1]/div/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr

rows = 1 +len(driver.find_elements_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/div/sal-components-funds-portfolio/div/div[1]/div[2]/div/div[6]/div[1]/div/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr'))
cols=len(driver.find_elements_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/div/sal-components-funds-portfolio/div/div[1]/div[2]/div/div[6]/div[1]/div/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr[1]/td'))
list_1 = []
for i in range(1, rows):
    list_2 = []
    for j in range(1, cols+1):
        value = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/div/sal-components-funds-portfolio/div/div[1]/div[2]/div/div[6]/div[1]/div/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr['+str(i)+']/td['+str(j)+']').text
        print(value, end='     ')
        list_2.append(value)
    list_1.append(list_2)
    print()

Basic Materials     5.06     5.40     
Consumer Cyclical     10.03     10.46     
Financial Services     16.00     14.07     
Real Estate     2.91     2.42     
Communication Services     7.96     7.12     
Energy     5.26     4.21     
Industrials     9.52     14.27     
Technology     19.58     18.00     
Consumer Defensive     7.59     7.75     
Healthcare     12.92     12.37     
Utilities     3.17     3.94     


In [415]:
#put the list into a dataframe
df=pd.DataFrame(list_1)
df

,0,1,2
0,Basic Materials,5.06,5.40
1,Consumer Cyclical,10.03,10.46
2,Financial Services,16.00,14.07
3,Real Estate,2.91,2.42
4,Communication Services,7.96,7.12
5,Energy,5.26,4.21
6,Industrials,9.52,14.27
7,Technology,19.58,18.00
8,Consumer Defensive,7.59,7.75
9,Healthcare,12.92,12.37


In [416]:
#drop the last column, rename remaining columns

df2= df.drop([2],axis=1)
df2.rename(columns={0:'GICS Sector',1:'ACWI Weight'},inplace=True)

#rename sectors to match GICS sectors
df2['GICS Sector']=df2['GICS Sector'].replace({'Consumer Cyclical':'Consumer Discretionary','Basic Materials':'Materials','Financial Services':'Financials','Technology':'Information Technology','Consumer Defensive':'Consumer Staples'})
df2



,GICS Sector,ACWI Weight
0,Materials,5.06
1,Consumer Discretionary,10.03
2,Financials,16.00
3,Real Estate,2.91
4,Communication Services,7.96
5,Energy,5.26
6,Industrials,9.52
7,Information Technology,19.58
8,Consumer Staples,7.59
9,Healthcare,12.92


In [421]:
#get latest weightings for each holding

today = dt.datetime.today().strftime("%m.%d.%Y")
ClosePath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/GIF_historical_closes05.24.2022.xlsx'

weightdf=pd.read_excel(ClosePath,sheet_name='GIF_YTD_Perf',index_col=0)

#count number of columns. Today's market values are at index cols-6
cols=weightdf.shape[1]
cols

#dataframe should just be ticker and current value
weightdf=weightdf.iloc[:,[0,cols-6]]

#calculate weight of each position based on total portfolio MV
MVsum=weightdf['Latest_Total_Value'].sum()
weightdf['GIF_Weight']=weightdf['Latest_Total_Value']/MVsum
weightdf.tail()


,Ticker,Latest_Total_Value,GIF_Weight
29,RYE,33890.560425,0.044487
30,RYF,27845.439697,0.036552
31,VOX,11192.950211,0.014693
32,VRTX,39735.041626,0.052159
33,Cash_Balance,29536.320000,0.038772


In [423]:
#get path of cost basis file, which has GICS sectors listed
BasisPath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/GIF_CostBasis_asof05.24.2022.xlsx'

#get GICS sector from above file. Then dataframe should just be ticker and GICS sector
basisdf=pd.read_excel(BasisPath,sheet_name='GIF_attributes-cost',index_col=0)
basisdf=basisdf.iloc[:,[1,13]]

#combine the frames and match ticker w/ correct sector
combdf=pd.merge(weightdf,basisdf,on="Ticker",how='left')
combdf[['GICS_Sector']]=combdf[['GICS_Sector']].fillna("Cash")
combdf.rename(columns={'GICS_Sector':'GICS Sector','GIF_Weight':'GIF Weight'},inplace=True)

#remove excess whitespace in the GICS Sector fields
combdf['GICS Sector'] = combdf['GICS Sector'].str.strip()
combdf


,Ticker,Latest_Total_Value,GIF Weight,GICS Sector
0,AON,34053.781860,0.044702,Financials
1,AAPL,34388.200150,0.045141,Information Technology
2,BRK-B,24733.601074,0.032467,Financials
3,CZR,7118.099899,0.009344,Consumer Discretionary
4,CAT,18714.149445,0.024566,Industrials
5,CI,46632.958496,0.061214,Healthcare
6,DHI,14766.749382,0.019384,Consumer Discretionary
7,HON,12157.440430,0.015959,Industrials
8,RHS,13375.200195,0.017557,Consumer Staples
9,MU,21451.640884,0.028159,Information Technology


In [424]:
#calculate weight of each sector, drop the total value column

GIFsecdf=combdf.groupby('GICS Sector').sum()
GIFsecdf= GIFsecdf.drop(["Latest_Total_Value"],axis=1)
GIFsecdf


,GIF Weight
GICS Sector,
Cash,0.038772
Communication Services,0.046988
Consumer Discretionary,0.077510
Consumer Staples,0.079421
Energy,0.044487
Financials,0.161294
Healthcare,0.164266
Industrials,0.086940
Information Technology,0.165764


In [ ]:
#combine the GIF and ACWI weight dataframes

#combine the frames and match ticker w/ correct sector, use 0 for ACWI Cash weight, cast both weights as float,
#state weights as %, compute active weight as GIF-ACWI

activedf=pd.merge(GIFsecdf,df2,on="GICS Sector",how='left')

activedf['ACWI Weight'] = activedf['ACWI Weight'].fillna(0)
activedf['ACWI Weight'] = activedf['ACWI Weight'].astype(float)
activedf['GIF Weight']=activedf['GIF Weight']*100
activedf['GIF Weight'] = activedf['GIF Weight'].astype(float)
activedf['Active Weight']=activedf['GIF Weight']-activedf['ACWI Weight']
activedf



In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

writer = pd.ExcelWriter('C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/Active Weights/Active_Weights_asof05.25.2022.xlsx', engine='xlsxwriter')
activedf.to_excel(writer,sheet_name='Active Weights')
writer.save()
writer


#GIF_combined_cost['Report_data'] = dt.datetime.today().strftime("%m/%d/%Y")
#today = dt.datetime.today().strftime("%m.%d.%Y")
#writer = pd.ExcelWriter(f"Output_Data/GIF_CostBasis_asof{today}.xlsx", engine='xlsxwriter')


# Write each dataframe to a different worksheet.
#GIF_combined_attrib.to_excel(writer, sheet_name='GIF_attributes-cost')
#GIF_combined_cost.to_excel(writer, sheet_name='cost_raw')

# Close the Pandas Excel writer and output the Excel file.
#writer.save()
